In [1]:
import pandas as pd
import numpy as np

### 1. Load Data for test

In [2]:
df_train = pd.read_csv("../data/train_new.csv")
df_train_target = df_train.filter(items = ["id", "country_destination"])

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
df_test = pd.read_csv("../data/test_new.csv")

In [3]:
df_train_submit = pd.read_csv("../data/submit_train.csv")

In [4]:
df_train_submit["country"] = df_train_submit["country"].replace({1 : "NDF", 2 : "US", 3 : "other", 4 : "FR", 5 : "IT"})

In [5]:
df_train_submit.shape, df_train_target.shape

((1067255, 2), (213451, 2))

In [6]:
df_train_tmp = df_train.filter(items = ["id", "ac_month", "ac_day", "country_destination"])

In [27]:
df_test_tmp = df_test.filter(items = ["id", "ac_month", "ac_day"])

### 2. nDCG_score

In [7]:
def nDCG_score(df_submit, df_target):
    len_target =len(df_target)
    sum_dcg = 0;
    
    df_merge = df_submit.merge(df_target, how = "left", left_on = "id", right_on = "id")
    
    for idx in range(len_target):
        df_tmp = df_merge[idx * 5:(idx * 5) + 5]
        for i in range(5):
            if df_tmp["country"].iloc[i] == df_tmp["country_destination"].iloc[i]:
                sum_dcg += 1 / np.log2(i + 1 + 1)
        
    return sum_dcg / len_target

In [8]:
# test - nDCG_score
# nDCG_score(df_train_submit, df_train_target)

### 3. make submit list 

In [45]:
# train data 입력시 id, target 모두 넣어주어야 한다. 
# test data 입력시 id 모두 넣어주어야 한다.
def submit_kaggle(df_train, df_test, model, path, user_id = "id", target = "country_destination"):
    le = LabelEncoder()

    # feature, target 전처리
    y_train = le.fit_transform(df_train[target])
    X_train = df_train.drop([target, user_id], axis = 1)

    X_test_id = df_test[user_id] # id 따로 정의
    X_test = df_test.drop([user_id], axis = 1)

    # train data set으로 fitting
    model = model.fit(X_train, y_train)
    predic_proba = model.predict_proba(X_test)

    # submit data, predict data 생성
    df_submit = pd.DataFrame(columns=["id", "country"])

    ids = []
    cts = []
    for i in range(len(X_test_id)):
        idx = X_test_id.iloc[i]
        ids += [idx] * 5
        cts += le.inverse_transform(np.argsort(predic_proba[i])[::-1])[:5].tolist()

    df_submit["id"] = ids
    df_submit["country"] = cts

    df_submit.to_csv(path, index = False)

    !kaggle competitions submit -c airbnb-recruiting-new-user-bookings -f {path} -m "Message"

In [46]:
# test - submit_kaggle
submit_kaggle(df_train_tmp, df_test_tmp, GaussianNB(), "submit/submit.csv")

Successfully submitted to Airbnb New User Bookings

### 4. K-Fold nDCG

In [10]:
from sklearn.preprocessing import LabelEncoder

In [11]:
from sklearn.model_selection import KFold

In [12]:
from sklearn.naive_bayes import GaussianNB 

In [13]:
len(df_train_tmp)

213451

In [24]:
# input : train_data, model, target, k
# output : ndcg score of cross valiation
def cross_val_ndcg(df_train, model, user_id = "id", target = "country_destination" k = 5):
    # 준비
    cv = KFold(n_splits=k, shuffle=True, random_state=0)
    le = LabelEncoder()

    sum_ndcg = 0

    # feature, target 전처리
    y = df_train[target]
    y_label = le.fit_transform(y)
    X_id = df_train[user_id] # id 따로 정의
    X = df_train.drop([target, user_id], axis = 1)

    # data를 k개로 분리
    for idx_k, (train_index, test_index) in enumerate(cv.split(df_train)):
        ar_train_target_set = y.values[train_index]
        ar_train_feature_set = X.values[train_index]

        ar_test_target_set = y.values[test_index]
        ar_test_feature_set = X.values[test_index]
        ar_test_id = X_id[test_index]

    #     print(ar_train_feature_set.shape, ar_test_feature_set.shape, ar_test_id.shape)

        # train data set으로 fitting
        model = model.fit(ar_train_feature_set, ar_train_target_set)
        predic_proba = model.predict_proba(ar_test_feature_set)

        # submit data, predict data 생성
        df_test_submit = pd.DataFrame(columns=[user_id, 'country'])
        df_test_predict = pd.DataFrame(columns=[user_id, target])

        ids = []
        cts = []
        for i in range(len(ar_test_target_set)):
            idx = ar_test_id.iloc[i]
            ids += [idx] * 5
            cts += le.inverse_transform(np.argsort(predic_proba[i])[::-1])[:5].tolist()

        df_test_submit['id'] = ids
        df_test_submit['country'] = cts

        df_test_predict["id"] = ar_test_id
        df_test_predict[target] = ar_test_target_set

        # ndcg score 생성
        ndcg = nDCG_score(df_test_submit, df_test_predict)
        print("{}번째 스코어 : {}".format(idx_k+1, ndcg))
        sum_ndcg += ndcg

    return sum_ndcg / k

In [23]:
# test - cross_val_ndcg
cross_val_ndcg(df_train_tmp, GaussianNB(), k = 5)

0번째 스코어 : 0.8082220271659184
1번째 스코어 : 0.8063154466967162
2번째 스코어 : 0.8060016846117946
3번째 스코어 : 0.8063452896890533
4번째 스코어 : 0.8071058422312227


0.80679805807894101